This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/intro.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Expression,
    LeafSystem,
    LogVectorOutput,
    MakeVectorVariable,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    RigidTransform_,
    Simulator,
    SpatialInertia_,
    StartMeshcat,
    ToLatex,
    UnitInertia_,
)

from underactuated import ConfigureParser, ManipulatorDynamics, running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# Dynamics of the Double Pendulum

This first cell gives an example of how to run a simulation and animate the results.


In [ ]:
def double_pendulum_demo():
    # Set up a block diagram with the robot (dynamics) and a visualization block.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    # Load the double pendulum from Universal Robot Description Format
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/double_pendulum.urdf")
    plant.Finalize()

    builder.ExportInput(plant.get_actuation_input_port())
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-2.8, xmax=2.8, ymin=-2.8, ymax=2.8)

    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)

    # Set the initial conditions
    context = simulator.get_mutable_context()
    context.SetContinuousState(
        [1.0, 1.0, 0.0, 0.0]
    )  # (theta1, theta2, theta1dot, theta2dot)
    diagram.get_input_port(0).FixValue(context, [0.0, 0.0])  # Zero input torques

    # Simulate
    simulator.AdvanceTo(10.0)

    # Plot the results
    plt.figure()
    fields = ["shoulder", "elbow"]
    log = logger.FindLog(context)
    for i in range(2):
        plt.subplot(2, 1, i + 1)
        plt.plot(log.sample_times(), log.data()[(i, i + 2), :].transpose())
        plt.legend(["position", "velocity"])
        plt.xlabel("t")
        plt.ylabel(fields[i])
        plt.grid(True)
    display(mpld3.display())


double_pendulum_demo()

**NOTE** Make sure you open up the meshcat window (the url is output in your link above) to see the animation.

It's worth taking a peek at the [file that describes the robot](https://github.com/RussTedrake/underactuated/blob/master/underactuated/models/double_pendulum.urdf). URDF and SDF are two of the standard formats, and they can be used to describe even very complicated robots (like the Boston Dynamics humanoid).

## Inspecting the dynamics (the manipulator equations)

We can also use Drake to evaluate the manipulator equations.  First we will evaluate the manipulator equations for a particular robot (with numerical values assigned for mass, link lengths, etc) and for a particular state of the robot.


In [ ]:
plant = MultibodyPlant(time_step=0)
parser = Parser(plant)
ConfigureParser(parser)
parser.AddModelsFromUrl("package://underactuated/models/double_pendulum.urdf")
plant.Finalize()

# Evaluate the dynamics numerically
q = [0.1, 0.1]
v = [1, 1]
(M, Cv, tauG, B, tauExt) = ManipulatorDynamics(plant, q, v)
display(Markdown("$M = " + ToLatex(M, precision=2) + "$"))
display(Markdown("$Cv = " + ToLatex(Cv, precision=2) + "$"))
display(Markdown("$\\tau_G = " + ToLatex(tauG, precision=2) + "$"))
display(Markdown("$B = " + ToLatex(B, precision=2) + "$"))
display(Markdown("$\\tau_{ext} = " + ToLatex(tauExt, precision=2) + "$"))

Drake is also fairly unique in supporting symbolic computation (with floating point coefficients).  Here is an example of printing out the symbolic dynamics of the double pendulum.  (If you've ever written out the equations of your robot, you know they get complicated quickly!)

In [ ]:
# Evaluate the dynamics symbolically


def SymbolicManipulatorEquations():
    plant = MultibodyPlant(time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/double_pendulum.urdf")
    plant.Finalize()

    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()

    # State variables
    q = MakeVectorVariable(2, "q")
    v = MakeVectorVariable(2, "v")
    vd = MakeVectorVariable(2, "\dot{v}")

    # Parameters
    m = MakeVectorVariable(2, "m")
    l = MakeVectorVariable(2, "l")

    # TODO: pending https://github.com/RobotLocomotion/drake/issues/17245
    # g = Variable("g")
    # sym_plant.mutable_gravity_field().SetGravityVector(sym_context, [0, 0, -g])

    upper_arm = sym_plant.GetBodyByName("upper_arm")
    inertia = SpatialInertia_[Expression](
        m[0],
        [0, 0, -l[0]],
        UnitInertia_[Expression](l[0] * l[0], l[0] * l[0], 0),
    )
    upper_arm.SetSpatialInertiaInBodyFrame(sym_context, inertia)
    lower_arm = sym_plant.GetBodyByName("lower_arm")
    inertia = SpatialInertia_[Expression](
        m[1],
        [0, 0, -l[1]],
        UnitInertia_[Expression](l[1] * l[1], l[1] * l[1], 0),
    )
    lower_arm.SetSpatialInertiaInBodyFrame(sym_context, inertia)
    elbow_frame = sym_plant.GetJointByName("elbow").frame_on_parent()
    elbow_frame.SetPoseInParentFrame(
        sym_context, RigidTransform_[Expression]([0, 0, -l[0]])
    )
    (M, Cv, tauG, B, tauExt) = ManipulatorDynamics(sym_plant, q, v, sym_context)
    display(Markdown("$M = " + ToLatex(M, precision=2) + "$"))
    display(Markdown("$Cv = " + ToLatex(Cv, precision=2) + "$"))
    display(Markdown("$\\tau_G = " + ToLatex(tauG, precision=2) + "$"))
    display(Markdown("$B = " + ToLatex(B, precision=2) + "$"))
    display(Markdown("$\\tau_{ext} = " + ToLatex(tauExt, precision=2) + "$"))


SymbolicManipulatorEquations()

We use a similar mechanisms to support automatic differentiation; we'll see examples of that soon!

# Feedback Cancellation of the Double Pendulum

Let's say that we would like our simple double pendulum to act like a
simple single pendulum (with damping), whose dynamics are given by:
$$\begin{aligned} \ddot \theta_1 &= -\frac{g}{l}\sin\theta_1 -b\dot\theta_1 \\ \ddot\theta_2 &= 0. \end{aligned}$$

This is easily achieved using
(Note that our chosen dynamics do not actually stabilize $\theta_2$ -- this detail was left out for clarity, but would be necessary for any real
implementation.) 
$${\bf u}  = {\bf B}^{-1}\left[ {\bf C}\dot{{\bf q}} - {\bf \tau}_g + {\bf M}\begin{bmatrix} -\frac{g}{l}s_1 - b\dot{q}_1 \\ 0 \end{bmatrix} \right].$$

Since we are embedding a nonlinear dynamics (not a linear one), we refer
to this as "feedback cancellation", or "dynamic inversion".  This idea can,
and does, make control look easy - for the special case of a fully-actuated
deterministic system with known dynamics.  For example, it would have been
just as easy for me to invert gravity. Observe that the control derivations
here would not have been any more difficult if the robot had 100 joints.

## Acting like a single pendulum

First we implement our simple controller as a system that takes the pendulum state in, and outputs the motor torque.

In [ ]:
class Controller(LeafSystem):
    """Defines a feedback controller for the double pendulum.

    The controller applies torques at the joints in order to:

    1) cancel out the dynamics of the double pendulum,
    2) make the first joint swing with the dynamics of a single pendulum, and
    3) drive the second joint towards zero.

    The magnitude of gravity for the imposed single pendulum dynamics is taken
    as a constructor argument.  So you can do fun things like pretending that
    gravity is zero, or even inverting gravity!
    """

    def __init__(self, multibody_plant, gravity):
        # 4 inputs (double pend state), 2 torque outputs.
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("double_pend_state", 4)
        self.DeclareVectorOutputPort("torque", 2, self.DoCalcVectorOutput)
        self.plant = multibody_plant
        self.g = gravity

    def DoCalcVectorOutput(self, context, torque):
        double_pend_state = self.get_input_port(0).Eval(context)
        # Extract manipulator dynamics.
        q = double_pend_state[:2]
        v = double_pend_state[-2:]
        (M, Cv, tauG, B, tauExt) = ManipulatorDynamics(self.plant, q, v)

        # Desired pendulum parameters.
        length = 2.0
        b = 0.1

        # Control gains for stabilizing the second joint.
        kp = 1
        kd = 0.1

        # Cancel double pend dynamics and inject single pend dynamics.
        torque.SetFromVector(
            Cv
            - tauG
            - tauExt
            + M.dot(
                [
                    self.g / length * np.sin(q[0]) - b * v[0],
                    -kp * q[1] - kd * v[1],
                ]
            )
        )


def simulate(gravity=-9.8):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    # Load the double pendulum from Universal Robot Description Format
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/double_pendulum.urdf")
    plant.Finalize()

    controller = builder.AddSystem(Controller(plant, gravity))
    builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0), plant.get_actuation_input_port())

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-2.8, xmax=2.8, ymin=-2.8, ymax=2.8)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)

    # Set the initial conditions
    context = simulator.get_mutable_context()
    context.SetContinuousState((1.0, 0.0, 0.2, 0.0))  # (θ₁, θ₂, θ̇₁, θ̇₂)

    # Simulate
    simulator.AdvanceTo(3.0)

If we simulate this system with the default parameters (gravity = -9.8m/s), then our double pendulum acts like a single pendulum.

In [ ]:
simulate()

**NOTE** Make sure you open up the meshcat window (the url is output in your link above) to see the animation.

But if we've gone this far, we could have replaced the dynamics with almost anything.  For instance, with a simple change, we can use feedback cancellation to invert gravity!

In [ ]:
simulate(gravity=9.8)